<a href="https://colab.research.google.com/github/udaylunawat/Whats-this-rock/blob/main/notebooks/04_sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{intro-colab} -->

# Hyperparameter Tuning using Wandb Sweeps
# Wandb Sweeps Setup

In [ ]:
#@title Wandb mode
import os
mode = 'online' #@param {type:"string"}
os.environ['WANDB_MODE'] = mode

if 'WANDB_API_KEY' not in os.environ:
    if os.environ['WANDB_MODE'] == 'online':
        from getpass import getpass
        secret = getpass('Enter WandB API Key: ')
        os.environ['WANDB_API_KEY'] = secret
    else:
        print("WandB Offline!")

In [ ]:
#@title Setup
!rm -rf /content/Whats-this-rock/
!git clone https://github.com/udaylunawat/Whats-this-rock.git
# !git clone -b hydra https://github.com/udaylunawat/Whats-this-rock.git
%cd /content/Whats-this-rock/

!nvidia-smi
!sh src/scripts/setup.sh

In [ ]:
!git pull

In [ ]:
%%writefile configs/sweep.yaml
method: bayes
metric:
  goal: minimize
  name: val_loss
parameters:
  wandb.use:
    value: True
  wandb.mode:
    value: online
  wandb.project:
    value: Whats-this-rockv18
  notes:
    value: ""
  seed:
    values: [1]
  lr:
    values: [1e-2, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5]
  lr_decay_steps:
    distribution: uniform
    min: 10
    max: 10000
  lr_schedule:
    values:
      - cosine_decay_restarts
  epochs:
    value: 75
  data_path:
    value: data/4_tfds_dataset/
  dataset_id:
    values:
      - [1, 2, 3, 4]
  augmentation:
    values: [kerascv]
  class_weights:
    values: [True, False]
  optimizer:
    values: [adam]
  loss:
    values: [categorical_crossentropy]
  metrics:
    value: ["accuracy"]
  batch_size:
    value: 64
  num_classes:
    value: 7
  train_split:
    values:
      - 0.75
  image_size:
    value: 224
  image_channels:
    value: 3
  sampling:
    values: [None]
  backbone:
    values: [resnet]
  use_pretrained_weights:
    values: [True]
  trainable:
    values: [False, True]
  last_layers:
    distribution: int_uniform
    min: 1
    max: 50
  custom_callback:
    values: [False]
  preprocess:
    values: [True, False]
  dropout_rate:
    values: [0.3, 0.5]
  monitor:
    values: ["val_loss"]
  earlystopping.use:
    values: [False]
  earlystopping.patience:
    values: [10]
  reduce_lr.use:
    values: [False]
  reduce_lr.factor:
    values: [.9]
  reduce_lr.patience:
    values: [1]
  reduce_lr.min_lr:
    values: [1e-6]
  save_model:
    value: False

program: src/models/train.py
command:
  - ${env}
  - python
  - ${program}
  - ${args_no_hyphens}


In [ ]:
!wandb sweep \
    --project Whats-this-rockv17 \
    --entity udaylunawat \
    configs/sweep.yaml

In [ ]:
!wandb agent udaylunawat/Whats-this-rockv17/xrjlfpy0